In [2]:
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv(), override=True)

True

In [3]:
from langchain_community.llms import Ollama

llm = Ollama(model ="llama2", temperature=0)

In [4]:
# llm.invoke("What is LangGraph?")

In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_path=("https://www.langchain.com/langsmith"),
)
docs = loader.load()

In [6]:
from langchain_community.document_loaders import CSVLoader

csvloader = CSVLoader(file_path="test.csv")
csvdoc = csvloader.load()

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True,
)

In [8]:
all_splits = text_splitter.split_documents(docs)

In [9]:
from langchain_community.embeddings import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="nomic-embed-text"
)

In [10]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents = all_splits,
    embedding = embedding,
)

In [11]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":6}
)

In [12]:
retriever.get_relevant_documents(
    "What is LangGraph"
)

[Document(page_content='Continuously track qualitative characteristics of any live application, and spot issues in real-time with LangSmith monitoring.Add observability to strengthen your applications.Request a demoSign UpOne-click deployEasily deploy your applications with hosted LangServe. Get built in parallelization, fallbacks, batch, streaming, and async support, coupled with familiar LangSmith visibility and a built-in playground.Explore LangServe\n\n\nMonitor cost, latency, quality.See what’s happening with your production application, so you can take action when needed or rest assured while your chains and agents do the hard work.Go to Docs\n\n\nUser feedback collection\n\n\n\n\n\n\n\nAdvanced filtering\n\n\n\n\n\n\n\nOnline \u2028auto-evaluation\n\n\n\n\n\n\n\nCost tracking\n\n\n\n\n\n\n\nInspect anomalies and errors\n\n\n\n\n\n\n\nSpot latency spikes\n\n\n\n\n\n\n\nUser feedback collectionAdvanced filteringOnline \u2028auto-evaluationCost trackingInspect anomalies and errorsS

In [13]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [14]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
rag_chain.invoke("What is LangSmith used for ")

"LangSmith is a platform for developing, collaborating, testing, deploying, and monitoring LLM applications. It provides a unified DevOps experience for users, allowing them to log traces, create datasets, run evals, and monitor their applications in real-time. LangSmith collects traces from various sources, including the LangChain and other applications, and stores them in GCP us-central-1. The platform is designed to provide full visibility into what's happening in users' applications, allowing them to debug and improve their applications easily.\n\nTo answer your question, LangSmith does not train on the data that users send it. Instead, it collects and stores traces for monitoring and debugging purposes. Additionally, LangSmith allows users to specify the percentage of traces they want to send to the platform, starting from SDK version 0.0.84 (Python) and 0.0.64 (JS). This feature enables users to control the amount of data they want to send to LangSmith while still benefiting from

In [18]:
# Chains in Langchain

In [ ]:
tools []

In [19]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()

python_repl.run("print(1+1)")

Python REPL can execute arbitrary code. Use with caution.


'2\n'

In [22]:
# You can create the tool to pass to an agent
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

In [23]:
from langchain.agents import load_tools

tool_names = [python_repl]
tools = load_tools(tool_names)

TypeError: unhashable type: 'PythonREPL'

In [ ]:
from langchain import LLMMathChain
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool, DuckDuckGoSearchRun

# llm_math_chain = LLMMathChain(llm=llm, verbose=True)

math_tool = Tool.from_function(
    func=llm.run,
    name="Calculator",
    description="Useful for when you are asked to perform math calculations"
)

tools = [search_tool, math_tool]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)